## Product Analytics and Recommendations on Online Retail data

In this file, we are going to focus on developing a recommendation model that will recommend products based on previous purchase orders and their quantities.

#### 1. Importing libraries and data

In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity

When loading the dataset, we filter all rows in column ‘Quantity’ that are greater than 0. This means that we now have a data record that contains at least one purchased product per line.

In [3]:
df = pd.read_excel('./data/Online Retail.xlsx', sheet_name='Online Retail')
df = df.loc[df['Quantity'] > 0]
df = df.dropna(subset=['CustomerID'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
customer_item_matrix = df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity', aggfunc='sum')

print('Shape of matrix: ' + str(customer_item_matrix.shape))

customer_item_matrix

Shape of matrix: (4339, 3665)


StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
12349.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we are going to encode this data 0-1. Hereby the value 1 means that the given product was purchased by the given customer and the value of 0 means that the given product was never purchased by the given customer.

In [6]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12349.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18281.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18282.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 2. Collaborative Filtering

One of the biggest prerequisites of a collaborative filtering system is the availability of data of past activity. Here we are going to explore two approches to building a product recommender:
1. user-based
2. item-based

In the first approach (user-based) we compute similarities between users based on their item purchase history. In the second approach (item-based) we compute similarities between items based in which items are often bought together with which other items.

The intentions behind it are the following:

In the first variant (user-based) it is checked whether customers buy many items together except for a few. It is possible that the items in which they differ are also interesting for the other customer.

#### 2.1.1 User-based Collaborative Filtering

'''User-to-User Similarity Matrix'''

In [9]:
user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_user_sim_matrix.columns = customer_item_matrix.index
user_user_sim_matrix['CustomerID'] = customer_item_matrix.index
user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')
user_user_sim_matrix

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.063022,0.046130,0.047795,0.038484,0.0,0.025876,0.136641,0.094742,...,0.0,0.029709,0.052668,0.000000,0.032844,0.062318,0.000000,0.113776,0.109364,0.012828
12348.0,0.0,0.063022,1.000000,0.024953,0.051709,0.027756,0.0,0.027995,0.118262,0.146427,...,0.0,0.064282,0.113961,0.000000,0.000000,0.000000,0.000000,0.000000,0.170905,0.083269
12349.0,0.0,0.046130,0.024953,1.000000,0.056773,0.137137,0.0,0.030737,0.032461,0.144692,...,0.0,0.105868,0.000000,0.000000,0.039014,0.000000,0.000000,0.067574,0.137124,0.030475
12350.0,0.0,0.047795,0.051709,0.056773,1.000000,0.031575,0.0,0.000000,0.000000,0.033315,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044866,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280.0,0.0,0.062318,0.000000,0.000000,0.000000,0.000000,0.0,0.041523,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.105409,1.000000,0.119523,0.000000,0.000000,0.000000
18281.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.049629,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.119523,1.000000,0.000000,0.046613,0.000000
18282.0,0.0,0.113776,0.000000,0.067574,0.000000,0.037582,0.0,0.000000,0.160128,0.079305,...,0.0,0.174078,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.017800,0.000000


As we can see customer 12348 has the greatest similarity to customer 12347 than customer 12349. Let’s pick customer 12347 to see the further proceeding. As first step we are going to rank the most similar customers to this customer (12347). Hereby we use the pairwise cosin similarity.

In [10]:
user_user_sim_matrix.loc[12347.0].sort_values(ascending=False)

CustomerID
12347.0    1.000000
17396.0    0.301694
13324.0    0.256186
13759.0    0.241355
16855.0    0.226050
             ...   
15402.0    0.000000
15400.0    0.000000
15397.0    0.000000
15395.0    0.000000
15299.0    0.000000
Name: 12347.0, Length: 4339, dtype: float64

As we can see, customer 17396 (our current target customer) is most similar to customer 12347.

Now we are going to identify the items that the customer 12347 and 17396 have already bought. Then, we are going to find the products that the target customer 17396 has not purchased, but customer 12347 has. We hope that the target customer can also use these products and will buy them if we recommend them to him.

'''Here we are going to look for which items customer 12347 has purchased in the past'''

In [13]:
items_bought_by_12347 = set(customer_item_matrix.loc[12347.0].iloc[customer_item_matrix.loc[12347.0].to_numpy().nonzero()].index)

'''Here we are going to look for which items target customer 17396 has purchased in the past'''

In [14]:
items_bought_by_17396 = set(customer_item_matrix.loc[17396.0].iloc[customer_item_matrix.loc[17396.0].to_numpy().nonzero()].index)

In [15]:
items_to_recommend_to_17396 = items_bought_by_12347 - items_bought_by_17396

print('Number of items that can be recommended to the target customer: ' + str(len(items_to_recommend_to_17396)))

Number of items that can be recommended to the target customer: 88


'''Get descriptions of items to recommend'''

In [21]:
df_items_to_recommend_to_17396 = df.loc[df['StockCode'].isin(items_to_recommend_to_17396), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').reset_index()

df_items_to_recommend_to_17396

,StockCode,Description
0,84969,BOX OF 6 ASSORTED COLOUR TEASPOONS
1,21791,VINTAGE HEADS AND TAILS CARD GAME
2,21035,SET/2 RED RETROSPOT TEA TOWELS
3,21731,RED TOADSTOOL LED NIGHT LIGHT
4,22492,MINI PAINT SET VINTAGE
...,...,...
90,23480,MINI LIGHTS WOODLAND MUSHROOMS
91,23497,CLASSIC CHROME BICYCLE BELL
92,23497,CLASSIC CROME BICYCLE BELL
93,71477,COLOURED GLASS STAR T-LIGHT HOLDER


In [17]:
df_items_to_recommend_to_17396[df_items_to_recommend_to_17396['StockCode'] == 23271]

,StockCode,Description
80,23271,CHRISTMAS TABLE CANDLE SILVER SPIKE
82,23271,CHRISTMAS TABLE SILVER CANDLE SPIKE


#### 2.1.2 Item-based Collaborative Filtering

In [22]:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
item_item_sim_matrix.columns = customer_item_matrix.T.index
item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
item_item_sim_matrix

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.000000,0.0,0.000000,0.0,0.000000,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.000000,0.0,0.000000,0.0,0.000000,0.036955,0.0,0.070185,0.049752,0.021877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2,0.029361,0.000000,0.000000,0.000000,0.036955,0.019360,0.055989,0.000000,0.000000,0.039996,...,0.000000,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.026196,0.000000,0.020413
DOT,0.000000,0.000000,0.000000,0.000000,0.000000,0.104257,0.150756,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
M,0.066915,0.016182,0.070535,0.070535,0.070185,0.066184,0.106335,0.063801,0.059013,0.086089,...,0.049875,0.0,0.040723,0.0,0.089220,0.026196,0.0,1.000000,0.000000,0.077539


The interpretation of the matrix which item is more similar to another item etc. follows the same principle as I explained with the user_user_sim_matrix.

The following procedure is also very similar.

First we select a product (let’s say StockCode=23166) that a customer has just bought as an example. Then we will identify the most similar products to this product just purchased. With the knowledge we have gained, we can then approach the customer and suggest similar products to the product he has just purchased.

In [24]:
top_10_similar_items_for_23166 = list(item_item_sim_matrix.loc[23166].sort_values(ascending=False).iloc[:10].index)

top_10_similar_items_for_23166

[23166, 23165, 23167, 22993, 23307, 22722, 22720, 22666, 23243, 22961]

Now we going to request the descriptions for the respective items again.

In [25]:
df_items_similar_to_23166 = df.loc[df['StockCode'].isin(top_10_similar_items_for_23166), 
    ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[top_10_similar_items_for_23166]

df_items_similar_to_23166

,Description
StockCode,
23166,MEDIUM CERAMIC TOP STORAGE JAR
23165,LARGE CERAMIC TOP STORAGE JAR
23167,SMALL CERAMIC TOP STORAGE JAR
22993,SET OF 4 PANTRY JELLY MOULDS
23307,SET OF 60 PANTRY DESIGN CAKE CASES
22722,SET OF 6 SPICE TINS PANTRY DESIGN
22720,SET OF 3 CAKE TINS PANTRY DESIGN
22666,RECIPE BOX PANTRY YELLOW DESIGN
23243,SET OF TEA COFFEE SUGAR TINS PANTRY


In the first line we see the article that was just bought. Afterwards 9 more items are shown, which have a high similarity to the purchased item.

In this case the client has bought MEDIUM CERAMIC TOP STORAGE JAR. Maybe he is also interested in buying the large variant (line 2)